# Assignment 6 - Part B

#### Refer to this code: [COLABLINK](https://colab.research.google.com/drive/1uJZvJdi5VprOQHROtJIHy0mnY2afjNlx)
WRITE IT AGAIN SUCH THAT IT ACHIEVES
#### 99.4% validation/test accuracy (50/10k split, basically we are calling Validation dataset as test dataset itself)
#### Less than 20k Parameters
You can use anything from above you want. 
#### Less than 20 Epochs
#### Have used BN, Dropout,
#### (Optional): a Fully connected layer or, have used GAP. 
To learn how to add different things we covered in this session, you can refer to this code: https://www.kaggle.com/enwei26/mnist-digits-pytorch-cnn-99 DONT COPY ARCHITECTURE, JUST LEARN HOW TO INTEGRATE THINGS LIKE DROPOUT, BATCHNORM, ETC.
This is a slightly time-consuming assignment, please make sure you start early. You are going to spend a lot of effort running the programs multiple times

In [1]:
#Import all necessary libraries
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

## Neural network architecture



1.   1st Convolution layer with 1 input and 16 output channels
2.   This is followed by a batch normalization and a max pool layer
3. A dropout layer is added after this
4. Another block of exact same layers (convolution, batch norm, max pool and dropout). This has 16 channels as input and 32 channels as output
5. Third convolution layer with 32 input channels and 32 output channels. This helps in increasing our receptive field while using small number of parameters
6. Final 1x1 convolution to reduce number of channels 32 to 16.
7. We use a ReLU activation after each of our convolulation layer
8. Finally one fully connected layer that converts 16 channels of 5x5 to 1x10 outputs.



In [13]:
class SimpleCNN(nn.Module):
  def __init__(self):
    super(SimpleCNN, self).__init__()
    self.conv1 = nn.Conv2d(1, 16, kernel_size=3)
    self.bn1 = nn.BatchNorm2d(16)
    self.dropout1 = nn.Dropout(0.01)
    self.conv2 = nn.Conv2d(16, 32, kernel_size=3)
    self.bn2 = nn.BatchNorm2d(32)
    self.dropout2 = nn.Dropout(0.01)
    self.conv3 = nn.Conv2d(32, 32, kernel_size=3)
    self.bn3 = nn.BatchNorm2d(32)
    self.dropout3 = nn.Dropout(0.01)
    self.conv1x1_reduce = nn.Conv2d(32, 16, kernel_size=1)
    self.fc = nn.Linear(16*5*5, 10)

  def forward(self, x):
    x = F.relu(self.bn1(self.conv1(x)))  # 26x26x16
    x = self.dropout1(x)
    x = F.relu(self.bn2(F.max_pool2d(self.conv2(x), 2))) # 12x12x32
    x = self.dropout2(x)
    x = F.relu(self.bn3(F.max_pool2d(self.conv3(x), 2))) # 5x5x32
    x = self.dropout3(x)
    #x = F.relu(self.bn4(F.max_pool2d(self.conv4(x), 2))) # 3x3x10
    
    x = self.conv1x1_reduce(x) # 5x5x16
    
    x = x.view(-1, 16*5*5)
    x = self.fc(x)
    x = x.view(-1, 10)
    return F.log_softmax(x, dim=1)




In [ ]:
!pip install torchsummary

In [14]:
#Creating model summary

from torchsummary import summary
use_cuda = torch.cuda.is_available() #Check if CUDA is available or not
device = torch.device("cuda" if use_cuda else "cpu") #Use CUDA if available
model = SimpleCNN().to(device) #Load model to device
summary(model, input_size=(1, 28, 28)) #Create summary if input image is 28X28

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
       BatchNorm2d-2           [-1, 16, 26, 26]              32
           Dropout-3           [-1, 16, 26, 26]               0
            Conv2d-4           [-1, 32, 24, 24]           4,640
       BatchNorm2d-5           [-1, 32, 12, 12]              64
           Dropout-6           [-1, 32, 12, 12]               0
            Conv2d-7           [-1, 32, 10, 10]           9,248
       BatchNorm2d-8             [-1, 32, 5, 5]              64
           Dropout-9             [-1, 32, 5, 5]               0
           Conv2d-10             [-1, 16, 5, 5]             528
           Linear-11                   [-1, 10]           4,010
Total params: 18,746
Trainable params: 18,746
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

## Creating Dataloader object for training and testing model

In [18]:
import random
import torchvision
torch.manual_seed(1)
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 128
# Load MNIST dataset with augmentations for training

transform_train = transforms.Compose([

                        transforms.GaussianBlur(kernel_size=5, sigma=(0.1, 2.0)),  # Apply Gaussian Blur
                        transforms.RandomAffine(degrees=20, translate=(0.1,0.1), scale=(0.9, 1.1)),
                        transforms.ColorJitter(brightness=0.2, contrast=0.2),
                        transforms.ToTensor(), 
                        transforms.Normalize((0.1307,), (0.3081,))
])

# Test dataset without augmentations
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
])


full_dataset = torchvision.datasets.MNIST(root="./data", train=True, download=True, transform=None)

#full_dataset = torchvision.datasets.MNIST(root="./data", train=True, download=True, transform=transform_train)


# Step 3: Split the dataset into 50k for training and 10k for testing
train_size = 50000
test_size = 10000
trainset, testset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

# Step 4: Create data loaders
batch_size = 128

# Step 4: Apply specific transforms to the train and test datasets
# Wrap the datasets with the appropriate transform
trainset.dataset.transform = transform_train
testset.dataset.transform = transform_test



# Load MNIST training and test datasets
#trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(full_dataset , batch_size=batch_size, shuffle=True)

# Total set of 70K. 60K for training and 10K for testing
#testset = torchvision.datasets.MNIST(root="./data", train=False, download=True, transform=transform_test)

#testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False)

# Check the sizes of the splits
print (f"Dataset sizes: Full dataset - {len(full_dataset)}, Train dataset - {len(trainset)}, Test dataset - {len(testset)}")
#print(f"Train dataset size: {len(trainset)}")
#print(f"Test dataset size: {len(testset)}")



Dataset sizes: Full dataset - 60000, Train dataset - 50000, Test dataset - 10000


## Creating training and testing methods

In [19]:
import matplotlib.pyplot as plt
from tqdm import tqdm



# Function to print model summary with layer details
def print_model_summary(model, input_size):
    model.to(device)  # Ensure the model is on the correct device
    summary(model, input_size)

print_model_summary(model, (1, 28, 28))  # MNIST images are 1 channel, 28x28 pixels
#criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

criterion = nn.NLLLoss()

#optimizer = optim.Adam(model.parameters(), lr=0.1)

def train(model, device, train_loader, optimizer, epoch):
    model.train() #Set model to train mode
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad() #Preventing gradient accumulation
        output = model(data)
        #loss = criterion(output, target) #Negative log likelihood loss
        loss = F.nll_loss(output, target)
        loss.backward() #Backpropagation. Weight calculation.
        optimizer.step() #Update parameter values
        pbar.set_description(desc= f'epoch={epoch} loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval() #Set model to test
    test_loss = 0
    correct = 0
    incorrect_samples = []  # To store incorrect predictions

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            #test_loss += criterion(output, target).item()  # sum up batch loss
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

            # Store incorrect predictions
            for i in range(data.size(0)):
                if pred[i].item() != target[i].item():
                    incorrect_samples.append((data[i].cpu(), target[i].item(), pred[i].item()))

    test_loss /= len(test_loader.dataset)

    accuracy = 100. * correct / len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), accuracy))
    
    # Display some incorrect predictions
    if incorrect_samples and False:
        num_display = min(10, len(incorrect_samples))  # Display up to 10 incorrect predictions
        fig, axes = plt.subplots(1, num_display, figsize=(15, 3))
        
        for i, (image, true_label, pred_label) in enumerate(incorrect_samples[:num_display]):
            image = image.squeeze(0)  # Remove channel dimension
            axes[i].imshow(image, cmap='gray')
            axes[i].set_title(f' T: {true_label}, P: {pred_label} ')
            axes[i].axis('off')
        
        plt.show()
    
    return accuracy

def train_model(num_epochs=20):

    show_images = True
    if show_images:
        all_images, all_labels = next(iter(torch.utils.data.DataLoader(trainset, batch_size=len(trainset))))
        indices = random.sample(range(len(all_images)), 5)

        # Step 4: Display the selected images
        plt.figure(figsize=(10, 5))
        for i, idx in enumerate(indices):
            image, label = all_images[idx], all_labels[idx]
            plt.subplot(1, 5, i + 1)
            plt.imshow(image.squeeze(), cmap="gray")
            plt.title(f"Label: {label.item()}")
            plt.axis("off")

        plt.tight_layout()
        plt.show()  # Show the plot without blocking code execution
        #plt.show(block=False)  # Show the plot without blocking code execution
        #plt.pause(2)  # Pause for 2 seconds
        #plt.close()  # Close the plot window


    for epoch in range(0, num_epochs): #Run for 18 epochs
        train(model, device, trainloader, optimizer, epoch)
    
    return model



def evaluate_model(testdata=1):
    print(f"Evaluating model: {testdata}", end="")
    if testdata == 1:
        return test(model, device, testloader)
    else:
        return test(model, device, trainloader)


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
       BatchNorm2d-2           [-1, 16, 26, 26]              32
           Dropout-3           [-1, 16, 26, 26]               0
            Conv2d-4           [-1, 32, 24, 24]           4,640
       BatchNorm2d-5           [-1, 32, 12, 12]              64
           Dropout-6           [-1, 32, 12, 12]               0
            Conv2d-7           [-1, 32, 10, 10]           9,248
       BatchNorm2d-8             [-1, 32, 5, 5]              64
           Dropout-9             [-1, 32, 5, 5]               0
           Conv2d-10             [-1, 16, 5, 5]             528
           Linear-11                   [-1, 10]           4,010
Total params: 18,746
Trainable params: 18,746
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

## Train and test the model

In [20]:
# Initialize model, loss function, and optimizer
model = SimpleCNN().to(device)

optimizer = optim.SGD(model.parameters(), lr=0.15, momentum=0.9) #Using SGD optimizer


def train_and_test_model(num_epochs=20):
    for epoch in range(0, num_epochs): #Run for 18 epochs
        train(model, device, trainloader, optimizer, epoch)
        #print("Evaluating model: Training accuracy")
        #evaluate_model(testdata=0)
        print("Evaluating model: Test accuracy")
        test(model, device, testloader)

train_and_test_model(num_epochs=20)

epoch=0 loss=0.05186476185917854 batch_id=468: 100%|██████████| 469/469 [00:30<00:00, 15.53it/s] 


Evaluating model: Test accuracy

Test set: Average loss: 0.0466, Accuracy: 9846/10000 (98.46%)



epoch=1 loss=0.031148681417107582 batch_id=468: 100%|██████████| 469/469 [00:30<00:00, 15.43it/s] 


Evaluating model: Test accuracy

Test set: Average loss: 0.0448, Accuracy: 9847/10000 (98.47%)



epoch=2 loss=0.0026431765872985125 batch_id=468: 100%|██████████| 469/469 [00:29<00:00, 15.83it/s]


...

Evaluating model: Test accuracy

Test set: Average loss: 0.0089, Accuracy: 9972/10000 (99.72%)



epoch=18 loss=0.0005756766768172383 batch_id=468: 100%|██████████| 469/469 [00:29<00:00, 16.09it/s] 


Evaluating model: Test accuracy

Test set: Average loss: 0.0072, Accuracy: 9979/10000 (99.79%)



epoch=19 loss=4.747032289742492e-05 batch_id=468: 100%|██████████| 469/469 [00:29<00:00, 16.09it/s] 


Evaluating model: Test accuracy

Test set: Average loss: 0.0114, Accuracy: 9959/10000 (99.59%)

